In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import pandas_datareader as pdr
import talib
%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

In [126]:
# start date 
start = datetime.datetime(2017,1,25)
#end date
end = datetime.datetime(2021,1,26)

In [136]:
class backtest_RSI:
    
    def __init__(self,ticker,start,end,RSI_range,TP_level,SL_level):
        
        #Create Instance attributes
        self.ticker = ticker
        self.start = start
        self.end = end
        self.RSI_range = RSI_range
        self.TP_level = TP_level
        self.SL_level = SL_level
    
        #Create Instance Methods
        self.fetch_data()
        self.indicators()
        self.process()
        self.signals()
        self.positions()
        self.returns()
        
    def fetch_data(self):
        self.df = pdr.get_data_yahoo(self.ticker,self.start,self.end)
        
    def indicators(self):
        
        self.df['RSI'] = talib.RSI(self.df['Adj Close'],timeperiod = self.RSI_range)
        
    def process(self):
        self.df.drop(['High', 'Low', 'Open', 'Close', 'Volume'],axis=1,inplace=True)
        self.df.dropna(inplace=True,axis=0)
        self.df = self.df.reset_index()
        
    def signals(self):
        
        self.df['entry_price'] = 0
        self.df['RSI_signal'] = 0
        
        n = self.df.shape[0]
        a=1
        b=0
        
        for i in range(0,n):
            
            if self.df['RSI'].loc[i] <= 30 and a == 1:
                self.df['RSI_signal'].loc[i] = 1
                self.df['entry_price'].loc[i] = self.df['Adj Close'].loc[i]
                
                a=0
                b=1
                
            if b==1 and self.df['RSI'].loc[i] >=70:
                self.df['RSI_signal'].loc[i] = -1
                
                a=1
                b=0
                
        self.df['take_profit_price'] = self.df['entry_price']*(1+ (self.TP_level/100))
        self.df['stop_loss_price'] = self.df['entry_price']*(1- (self.SL_level/100))
        
        self.df['take_profit_price']=self.df['take_profit_price'].replace(to_replace=0,method='ffill')
        self.df['stop_loss_price']=self.df['stop_loss_price'].replace(to_replace=0,method='ffill')
        
        for i in range(0,n):
            if self.df['Adj Close'].loc[i] < self.df['stop_loss_price'].loc[i] or self.df['Adj Close'].loc[i] > self.df['take_profit_price'].loc[i]:
                self.df['RSI_signal'].loc[i] = -1
                
        self.df['RSI_signal']=self.df['RSI_signal'].replace(to_replace=0,method='ffill')
        
    def positions(self):
        
        self.df['positions']= self.df['RSI_signal'].replace(to_replace=-1,value=0)
        self.df['positions'] = self.df['positions'].shift()
        
    def returns(self):
        
        self.df['BnH_Returns'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift())
        self.df['Strategy_Returns'] = self.df['BnH_Returns']*self.df['positions']
        Total_Returns = self.df['Strategy_Returns'].sum()
        print('Total Strategy Returns:', self.df['Strategy_Returns'].sum())
        return Total_Returns
    
        
    
                
                
        

In [138]:
indice_list=['^NSEI','SPY','HSI','^CNXIT','^NSEBANK','^HSI','^GSPC']

RSI =[]
indices=[]
Strategy_Returns =[]
Take_Profit=[]
Stop_Loss =[]

for i in indice_list:
    for j in range(8,21,1):
        for k in range(2,9):
            for l in range(1,6):
                print('For',i,j)
                a = backtest_RSI(i,start,end,j,0.07,0.02)
                indices.append(i)
                RSI.append(j)
                Take_Profit.append(k)
                Stop_Loss.append(l)
                Strategy_Returns.append(a.returns())

For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0.014362457524333204
For ^NSEI 8
Total Strategy Returns: 0.014362457524333204
Total Strategy Returns: 0

Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For ^NSEI 10
Total Strategy Returns: 0.07276627261923208
Total Strategy Returns: 0.07276627261923208
For

Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Strategy Returns: 0.0696953305838103
Total Strategy Returns: 0.0696953305838103
For ^NSEI 12
Total Stra

Total Strategy Returns: -0.04254240006756131
Total Strategy Returns: -0.04254240006756131
For ^NSEI 14
Total Strategy Returns: -0.04254240006756131
Total Strategy Returns: -0.04254240006756131
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.03028586362524164
For ^NSEI 15
Total Strategy Returns: -0.03028586362524164
Total Strategy Returns: -0.0

Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Strategy Returns: -0.024107062262239895
For ^NSEI 17
Total Strategy Returns: -0.024107062262239895
Total Str

...

Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007479111499114
Total Strategy Returns: 0.007479111499114
For SPY 13
Total Strategy Returns: 0.007478143958890342
Total

Total Strategy Returns: 0.0028778766740727105
Total Strategy Returns: 0.0028778766740727105
For SPY 15
Total Strategy Returns: 0.0028778766740727105
Total Strategy Returns: 0.0028778766740727105
For SPY 15
Total Strategy Returns: 0.0028779638081666593
Total Strategy Returns: 0.0028779638081666593
For SPY 15
Total Strategy Returns: 0.0028778766740727105
Total Strategy Returns: 0.0028778766740727105
For SPY 15
Total Strategy Returns: 0.0028778967211705187
Total Strategy Returns: 0.0028778967211705187
For SPY 16
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 16
Total Strategy Returns: -0.026482991630209597
Total Strategy Returns: -0.026482991630209597
For SPY 16
Total Strategy Returns: -0.026483082610523234
Total Strategy Returns: -0.026483082610523234
For SPY 16
Total Strategy Returns: -0.026482890645189668
Total Strategy Returns: -0.026482890645189668
For SPY 16
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.

Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.026483443381914794
Total Strategy Returns: -0.026483443381914794
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.02648331769198925
Total Strategy Returns: -0.02648331769198925
For SPY 18
Total Strategy Returns: -0.026483221485926706
Total Strategy Returns: -0.026483221485926706
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.026482949620783965
For SPY 18
Total Strategy Returns: -0.026482949620783965
Total Strategy Returns: -0.02

Total Strategy Returns: 0.00958265749027344
Total Strategy Returns: 0.00958265749027344
For SPY 20
Total Strategy Returns: 0.009582067621241967
Total Strategy Returns: 0.009582067621241967
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For SPY 20
Total Strategy Returns: 0.009582555611450784
Total Strategy Returns: 0.009582555611450784
For

RemoteDataError: No data fetched for symbol HSI using YahooDailyReader

In [133]:
Results = pd.DataFrame({'Indice':indices,'RSI range':RSI, 'Take Proft':Take_Profit,'Stop Loss':Stop_Loss,'Returns':Strategy_Returns})
Results.sort_values(by='Returns',ascending=False)

,Indice,RSI range,Returns
2,^NSEI,10,0.250072
1,^NSEI,9,0.244370
14,SPY,9,0.174671
3,^NSEI,11,0.151987
4,^NSEI,12,0.117282
0,^NSEI,8,0.114106
13,SPY,8,0.075531
20,SPY,15,0.074817
17,SPY,12,0.049601
12,^NSEI,20,0.042838
